In [1]:
!pip install ultralytics scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.3/949.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:

# Clone the BCCD dataset from GitHub
!git clone https://github.com/Shenggan/BCCD_Dataset

!mv BCCD_Dataset BCCD

!ls /content/BCCD

Cloning into 'BCCD_Dataset'...
remote: Enumerating objects: 800, done.
remote: Total 800 (delta 0), reused 0 (delta 0), pack-reused 800 (from 1)
Receiving objects: 100% (800/800), 7.39 MiB | 39.19 MiB/s, done.
Resolving deltas: 100% (378/378), done.
BCCD  dataset  example.jpg  export.py  LICENSE	plot.py  README.md  scripts  test.csv


In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
base_path = '/content/BCCD/BCCD/'
images_path = os.path.join(base_path, 'images')

# Create train/val/test directories
os.makedirs(os.path.join(images_path, 'train'), exist_ok=True)
os.makedirs(os.path.join(images_path, 'val'), exist_ok=True)
os.makedirs(os.path.join(images_path, 'test'), exist_ok=True)

# Get all image paths
image_folder = os.path.join(base_path, 'JPEGImages')
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.jpg')]

# Split dataset (80% train, 10% val, 10% test)
train_imgs, temp_imgs = train_test_split(image_paths, test_size=0.2, random_state=42)
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)

# Move images to respective folders
for img in train_imgs:
    shutil.move(img, os.path.join(images_path, 'train'))
for img in val_imgs:
    shutil.move(img, os.path.join(images_path, 'val'))
for img in test_imgs:
    shutil.move(img, os.path.join(images_path, 'test'))

print("Dataset successfully split into train, val, and test.")


Dataset successfully split into train, val, and test.


In [6]:
yaml_content = """
train: /content/BCCD/BCCD/images/train
val: /content/BCCD/BCCD/images/val
test: /content/BCCD/BCCD/images/test

nc: 3
names: ['Platelets', 'RBC', 'WBC']
"""

# Write data.yaml
yaml_path = '/content/BCCD/data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("data.yaml created successfully.")


data.yaml created successfully.


In [10]:
import os
import shutil

# Create labels folders if they don’t exist
for split in ['train', 'val', 'test']:
    os.makedirs(f'/content/BCCD/BCCD/labels/{split}', exist_ok=True)

# Move labels into correct folders
annotation_dir = '/content/BCCD/BCCD/Annotations'
for filename in os.listdir(annotation_dir):
    if filename.endswith('.txt'):
        img_name = filename.replace('.txt', '.jpg')
        if os.path.exists(f'/content/BCCD/BCCD/images/train/{img_name}'):
            shutil.move(f'{annotation_dir}/{filename}', '/content/BCCD/BCCD/labels/train/')
        elif os.path.exists(f'/content/BCCD/BCCD/images/val/{img_name}'):
            shutil.move(f'{annotation_dir}/{filename}', '/content/BCCD/BCCD/labels/val/')
        elif os.path.exists(f'/content/BCCD/BCCD/images/test/{img_name}'):
            shutil.move(f'{annotation_dir}/{filename}', '/content/BCCD/BCCD/labels/test/')

print("Labels moved to correct folders.")


Labels moved to correct folders.


In [23]:
import glob

# Look for XML annotation files
xml_files = glob.glob('/content/BCCD/**/*.xml', recursive=True)
print("Found XML annotations:", xml_files[:5])  # Show first 5 files



Found XML annotations: ['/content/BCCD/BCCD/Annotations/BloodImage_00353.xml', '/content/BCCD/BCCD/Annotations/BloodImage_00179.xml', '/content/BCCD/BCCD/Annotations/BloodImage_00366.xml', '/content/BCCD/BCCD/Annotations/BloodImage_00062.xml', '/content/BCCD/BCCD/Annotations/BloodImage_00037.xml']


In [24]:
import os

# Check for train images
train_images = os.listdir('/content/BCCD/BCCD/images/train')
print(f"Number of train images: {len(train_images)}")



Number of train images: 291


In [25]:
import os
import xml.etree.ElementTree as ET

# Paths
xml_folder = '/content/BCCD/BCCD/Annotations'
label_folder = '/content/BCCD/BCCD/labels/train'
os.makedirs(label_folder, exist_ok=True)

# Class mapping
class_map = {'Platelets': 0, 'RBC': 1, 'WBC': 2}

# Convert XML to YOLO format
for xml_file in os.listdir(xml_folder):
    if not xml_file.endswith('.xml'):
        continue

    # Parse XML
    tree = ET.parse(os.path.join(xml_folder, xml_file))
    root = tree.getroot()

    # Get image dimensions
    size = root.find('size')
    img_width = int(size.find('width').text)
    img_height = int(size.find('height').text)

    # Create YOLO label file
    txt_file = os.path.join(label_folder, xml_file.replace('.xml', '.txt'))

    with open(txt_file, 'w') as f:
        # Loop over each object in the image
        for obj in root.findall('object'):
            cls = obj.find('name').text
            if cls not in class_map:
                continue

            # Get bounding box
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)

            # Normalize coordinates (YOLO format)
            x_center = (xmin + xmax) / 2 / img_width
            y_center = (ymin + ymax) / 2 / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            # Write to YOLO format
            f.write(f"{class_map[cls]} {x_center} {y_center} {width} {height}\n")

print("Conversion complete! 🚀 YOLO labels are now in:", label_folder)



Conversion complete! 🚀 YOLO labels are now in: /content/BCCD/BCCD/labels/train


In [26]:
import os
train_labels = os.listdir('/content/BCCD/BCCD/labels/train')
print(f"Number of labels generated: {len(train_labels)}")


Number of labels generated: 364


In [27]:
results = model.train(data='/content/BCCD/data.yaml', epochs=50, imgsz=640)


Ultralytics 8.3.93 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/BCCD/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /content/BCCD/BCCD/labels/train... 291 images, 0 backgrounds, 0 corrupt: 100%|██████████| 291/291 [00:00<00:00, 1792.24it/s]

train: New cache created: /content/BCCD/BCCD/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/BCCD/BCCD/labels/val.cache... 0 images, 36 backgrounds, 0 corrupt: 100%|██████████| 36/36 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/BCCD/BCCD/labels/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...


50 epochs completed in 0.093 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 22.5MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.93 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.11it/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:491: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.05)
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.05)
/usr/local/lib/python3.11/dist-packages/numpy/core/_meth

                   all         36          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:654: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index


Speed: 0.4ms preprocess, 4.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train4


In [28]:
# Install Albumentations for data augmentation
!pip install albumentations

# Import necessary libraries
import cv2
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm


In [29]:
# Paths to images and labels
train_images_path = '/content/BCCD/BCCD/images/train'
train_labels_path = '/content/BCCD/BCCD/labels/train'

# Augmented output paths
augmented_images_path = '/content/BCCD/BCCD/images/augmented_train'
augmented_labels_path = '/content/BCCD/BCCD/labels/augmented_train'

# Create directories if they don’t exist
os.makedirs(augmented_images_path, exist_ok=True)
os.makedirs(augmented_labels_path, exist_ok=True)

# Define augmentations
transform = A.Compose([
    A.RandomRotate90(p=0.5),  # Rotate 90 degrees
    A.HorizontalFlip(p=0.5),  # Flip horizontally
    A.RandomBrightnessContrast(p=0.2),  # Adjust brightness/contrast
    A.CLAHE(p=0.2),
])

# Apply augmentation
for img_name in tqdm(os.listdir(train_images_path)):
    # Load image
    img_path = os.path.join(train_images_path, img_name)
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Apply transformations
    augmented = transform(image=image)
    augmented_image = augmented["image"]

    # Save augmented image
    aug_img_path = os.path.join(augmented_images_path, f'aug_{img_name}')
    cv2.imwrite(aug_img_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

    # Copy labels (YOLO format)
    label_name = img_name.replace('.jpg', '.txt')
    label_path = os.path.join(train_labels_path, label_name)
    aug_label_path = os.path.join(augmented_labels_path, f'aug_{label_name}')

    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            label_content = f.read()
        with open(aug_label_path, 'w') as f:
            f.write(label_content)

print(" Data Augmentation Completed!")


100%|██████████| 291/291 [00:01<00:00, 169.13it/s]

✅ Data Augmentation Completed!


In [30]:
# Update data.yaml path to include augmented images
yaml_content = """
train: /content/BCCD/BCCD/images/augmented_train
val: /content/BCCD/BCCD/images/val
test: /content/BCCD/BCCD/images/test

nc: 3
names: ['Platelets', 'RBC', 'WBC']
"""

yaml_path = '/content/BCCD/data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(" data.yaml updated successfully.")


✅ data.yaml updated successfully.


In [31]:
from ultralytics import YOLO

# Load YOLOv10 model
model = YOLO('yolov8n.pt')

results = model.train(data=yaml_path, epochs=50, imgsz=640)

print(" Retraining Completed!")


100%|██████████| 6.25M/6.25M [00:00<00:00, 70.6MB/s]


Ultralytics 8.3.93 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/BCCD/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /content/BCCD/BCCD/labels/augmented_train... 291 images, 0 backgrounds, 0 corrupt: 100%|██████████| 291/291 [00:00<00:00, 2126.33it/s]

train: New cache created: /content/BCCD/BCCD/labels/augmented_train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/BCCD/BCCD/labels/val.cache... 0 images, 36 backgrounds, 0 corrupt: 100%|██████████| 36/36 [00:00<?, ?it/s]

...


50 epochs completed in 0.079 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics 8.3.93 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.00it/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:491: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.05)
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:526: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.05)
/usr/local/lib/python3.11/dist-packages/numpy/core/_meth

                   all         36          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:654: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index


Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/train5
✅ Retraining Completed!


In [32]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00


In [34]:
import gradio as gr
from ultralytics import YOLO
from PIL import Image

# Load the trained model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Define prediction function
def predict(image):
    results = model(image)
    output_path = 'output.jpg'
    results.save(output_path)
    return output_path

# Create Gradio Interface
app = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Image(type="filepath"),
    title="BCCD Blood Cell Detection",
    description="Upload an image to detect Platelets, RBC, and WBC."
)

# Launch app
app.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bd559c2bfcacdb0dc8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [35]:
from google.colab import files
files.download('runs/detect/train/weights/best.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>